In [27]:

import os
import csv
import cv2
import matplotlib.pyplot as plt
from deepgaze.color_detection import RangeColorDetector
from deepgaze.mask_analysis import BinaryMaskAnalyser
import datetime
import numpy as np
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [28]:


# 提取特定区域的函数
def draw_polygon(image, points, color):
    points = np.array(points, dtype=np.int32)
    cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)

        
def drawROI(title, video_code, image, points, counter):
    output_file = "./pngFromVideo/" + video_code + "/" + title + "_" + str(counter).zfill(5) + ".jpg"
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file))
    # 创建一个与原始图像相同大小的黑色遮罩
    mask = np.zeros_like(image)

    # 将 forehead_points 转换为 NumPy 数组
    points = np.array(points, dtype=np.int32)

    # 在遮罩上绘制多边形
    cv2.fillPoly(mask, [points], color=(255, 255, 255))

    # 将遮罩应用到原始图像上
    result_image = cv2.bitwise_and(image, mask)

    # 显示结果图像
    ##cv2.imshow("Forehead Region", result_image)
    cv2.imwrite(output_file, result_image)
    ##cv2.destroyAllWindows()
    
def detectFace(video_code):
    # 設定方法
    BaseOptions = mp.tasks.BaseOptions
    FaceLandmarker = mp.tasks.vision.FaceLandmarker
    FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode
    ##video_path = "./Video/" + video_code +".mp4"
    video_path = "D:/2_NCU/1121_碩論/心律檢測影片/" + video_code +".mov"
    print(video_path)

    # 人臉偵測設定
    options = FaceLandmarkerOptions(
        base_options=BaseOptions(model_asset_path='model/face_landmarker.task'),
        running_mode=VisionRunningMode.IMAGE)

    forehead_indices = [66,69,108,151,337,299,296,336,9,107]
    nose_bridge_indices = [245,244,189,55,8,285,413,464,465,351,6,122]
    nose_indices = [ 6,196,236,219,220,5,440,439,456,419]
    nose_tip_indices = [5,220,19,440]
    left_cheek_indices = [33,7,163,144,145,153,154,155,133,243,244,233,121,142,203,61]
    right_cheek_indices = [263,249,390,373,374,380,381,382,362,463,464,453,350,371,375]
    upper_lip_indices = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 415]
    lower_lip_indices = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
    chin_indices = [200,208,175,428]
   

    # 執行人臉偵測
    with FaceLandmarker.create_from_options(options) as landmarker:
        counter = 0 
        counter = counter + 1
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        for frame_index in range(total_frames):
            ##if frame_index > 200:
            ##    break
            ret, frame = cap.read()             # 讀取影片的每一幀
            if not ret:
                print("Cannot receive frame")   # 如果讀取錯誤，印出訊息
                ##break
                continue
            
            h, w = frame.shape[:2]  # 畫面高度和寬度
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            face_landmarker_result = landmarker.detect(mp_image)

            if face_landmarker_result is None:
                continue

            face_landmarks_list = face_landmarker_result.face_landmarks
            annotated_image = np.copy(frame)

            for idx in range(len(face_landmarks_list)):
                face_landmarks = face_landmarks_list[idx]
                landmarks = [(int(landmark.x * w), int(landmark.y * h)) for landmark in face_landmarks]

                # 1. 繪製額頭
                forehead_points = [landmarks[i] for i in forehead_indices]
                drawROI("forehead", video_code, annotated_image, forehead_points, frame_index)
                draw_polygon(annotated_image, forehead_points, (255, 0, 0))
                

                # 2. 繪製左臉頰
                left_cheek_points = [landmarks[i] for i in left_cheek_indices]
                drawROI("leftCheek", video_code, annotated_image, left_cheek_points, frame_index)
                draw_polygon(annotated_image, left_cheek_points, (0, 255, 0))
                

                # 3. 繪製右臉頰
                right_cheek_points = [landmarks[i] for i in right_cheek_indices]
                drawROI("rightCheek", video_code, annotated_image, right_cheek_points, frame_index)
                draw_polygon(annotated_image, right_cheek_points, (0, 0, 255))

                # 4. 繪製人中
                nose_bridge_points = [landmarks[i] for i in nose_bridge_indices]
                drawROI("noseBridge", video_code, annotated_image, nose_bridge_points, frame_index)
                draw_polygon(annotated_image, nose_bridge_points, (60, 0, 255))
                # 5. 繪製鼻子
                nose_points = [landmarks[i] for i in nose_indices]
                drawROI("nose", video_code, annotated_image, nose_points, frame_index)
                draw_polygon(annotated_image, nose_points, (0, 60, 255))
                # 5. 繪製鼻頭
                nose_tip_points = [landmarks[i] for i in nose_tip_indices]
                drawROI("noseTip", video_code, annotated_image, nose_tip_points, frame_index)
                draw_polygon(annotated_image, nose_tip_points, (60, 60, 255))
                # 7. 繪製上嘴唇
                upper_lip_points = [landmarks[i] for i in upper_lip_indices]
                drawROI("upperLip", video_code, annotated_image, upper_lip_points, frame_index)
                draw_polygon(annotated_image, upper_lip_points, (255, 255, 0))

                # 8. 繪製下嘴唇
                lower_lip_points = [landmarks[i] for i in lower_lip_indices]
                drawROI("lowerLip", video_code, annotated_image, lower_lip_points, frame_index)
                draw_polygon(annotated_image, lower_lip_points, (0, 255, 255))
                # 9. 繪製下巴
                chin_points = [landmarks[i] for i in chin_indices]
                drawROI("chinLip", video_code, annotated_image, chin_points, frame_index)
                draw_polygon(annotated_image, chin_points, (0, 255, 255))

            ##cv2.imshow('Face Regions', annotated_image)  # 如果讀取成功，顯示該幀的畫面
            if cv2.waitKey(10) == ord('q'):  # 每一毫秒更新一次，直到按下 q 結束
                break
        cap.release()  # 所有作業都完成後，釋放資源
        cv2.destroyAllWindows()  # 結束所有視窗

In [29]:

def read_ulife_csv(file_in):
    ##detectFace("012_20240605_1133")
    with open(file_in, mode='r', newline='', encoding="utf-8") as file:
        reader = csv.DictReader(file)  # 使用DictReader读取带有表头的CSV文件
        for row in reader:
            tester_code = "{}_{}".format(row["編號"], datetime.datetime.strptime(row["量測日期時間"], "%Y/%m/%d %H:%M").strftime("%Y%m%d_%H%M"))
            
            detectFace(tester_code)
            print(tester_code)
##read_ulife_csv("ulife/uLifePlusApp.csv")
read_ulife_csv("ulife/uLifePlusApp_20240531_20240607.csv")

D:/2_NCU/1121_碩論/心律檢測影片/012_20240605_1133.mov
